Ask the user to choose a Brazilian stock and analyze the asset between January 1, 2015, and August 31, 2025. Return the following statistics to the user:

- Cumulative return year-on-year against the Ibovespa.
- Descriptive statistics of volatility, average returns, highest return, and lowest return per year.
- Maximum drawdown per year.
- 252-day correlation chart against the Ibovespa.
- Average annual financial volume traded by the company.

In [ ]:
# ------------------------------------------------------------
# EXERCÍCIO 88 — ANÁLISE COMPLETA DE UM ATIVO ESCOLHIDO PELO USUÁRIO
# ------------------------------------------------------------

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# 1) Escolher ação — input do usuário
ticker = input("Digite o ticker da ação brasileira (ex.: WEGE3.SA, PETR4.SA etc.): ")
# Ex.: WEGE3.SA, PETR4.SA etc.

# 2) Baixar dados do ativo e do Ibovespa no período especificado
inicio = "2015-01-01"
fim = "2025-08-31"

dados = yf.download([ticker, "^BVSP"], start=inicio, end=fim)
# baixa dados diários de PREÇO e VOLUME do ativo e do índice Ibovespa
# retorna um DataFrame multi-nível com colunas (Close, Volume etc.)

# 3) Extrair colunas Close e Volume
# preço de fechamento do ativo e do Ibov:
close = dados['Close']    # DataFrame com 2 colunas: ticker e ^BVSP
volume = dados['Volume']  # DataFrame com 2 colunas: ticker e ^BVSP

# renomear as colunas para ficar fácil
close.columns = ['Ativo','Ibov']
volume.columns = ['Ativo','Ibov']

# 4) Calcular retornos diários
retornos = close.pct_change().dropna()
# pct_change(): transforma preços em retornos percentuais diários
# dropna(): remove a 1ª linha sem retorno

# 5) Criar coluna 'ano'
retornos['ano'] = retornos.index.year
close['ano'] = close.index.year
volume['ano'] = volume.index.year

# ------------------------------------------------------------
# A) Retorno acumulado ano a ano contra o Ibovespa
# ------------------------------------------------------------
ret_acum_ativo = retornos.groupby('ano')['Ativo'].apply(lambda x: (1+x).prod()-1)
ret_acum_ibov = retornos.groupby('ano')['Ibov'].apply(lambda x: (1+x).prod()-1)

print("\n=== Retorno acumulado ano a ano ===")
ret_acum = pd.DataFrame({'Ativo':ret_acum_ativo, 'Ibov':ret_acum_ibov})
print((ret_acum*100).round(2))

# ------------------------------------------------------------
# B) Estatísticas descritivas por ano (vol, média, maior, menor)
# ------------------------------------------------------------
estatisticas = retornos.groupby('ano')['Ativo'].agg(
    media=lambda x: x.mean()*100,
    vol=lambda x: x.std()*np.sqrt(252),
    max=lambda x: x.max()*100,
    min=lambda x: x.min()*100
)
print("\n=== Estatísticas anuais do ativo ===")
print(estatisticas)

# ------------------------------------------------------------
# C) Máximo Drawdown por ano
# ------------------------------------------------------------
# preparar DataFrame para drawdown do ativo
df_dd = close[['Ativo']].copy()
df_dd.rename(columns={'Ativo':'Close'}, inplace=True)
df_dd['ano'] = df_dd.index.year
df_dd['maxima_do_ano'] = df_dd.groupby('ano')['Close'].cummax()
df_dd['quedas'] = df_dd['Close'] / df_dd['maxima_do_ano'] - 1
dd_max_ano = df_dd.groupby('ano')['quedas'].min()
print("\n=== Máximo Drawdown por ano (Ativo) ===")
print((dd_max_ano*100).round(2))

# ------------------------------------------------------------
# D) Gráfico de correlação de 252 dias contra o Ibovespa
# ------------------------------------------------------------
corr_movel = retornos['Ativo'].rolling(252).corr(retornos['Ibov'])
# rolling(252): janela móvel de 252 dias (~1 ano)
# .corr(...): correlação móvel do ativo vs Ibov
plt.figure(figsize=(12,5))
plt.plot(corr_movel, label='Correlação 252d vs Ibov')
plt.title(f'{ticker} — Correlação móvel 252 dias vs Ibovespa')
plt.axhline(0, ls='--', color='gray')
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

# ------------------------------------------------------------
# E) Volume financeiro médio anual negociado pela empresa
# ------------------------------------------------------------
# pegar volume negociado (ações) * preço médio diário para estimar volume financeiro
volume_financeiro_diario = volume['Ativo'] * close['Ativo']
# volume de ações * preço de fechamento = volume financeiro aproximado

volume_financeiro_anual_medio = volume_financeiro_diario.groupby(close['ano']).mean()
print("\n=== Volume financeiro médio anual (R$) ===")
print(volume_financeiro_anual_medio)
